## Analyzing CIA Factbook Data Using SQL
We are using data from the [CIA World Factbook](https://www.cia.gov/library/publications/the-world-factbook/) which contains statistics about all of the countries on Earth.  The data contains information on demographics such as:
* `population` - as of 2015
* `population_growth` - The annual population growth rate, as a percentage
* `area` - total land and water area

Connect to the database file

In [38]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

In [33]:
from sqlalchemy import  create_engine
 
engine = create_engine('sqlite:///factbook.db')
engine.connect()
print(engine)

Engine(sqlite:///factbook.db)


In [34]:
%%sql
SELECT * FROM sqlite_master WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [35]:
%%sql
SELECT * FROM facts limit 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


The table above is the first few rows of the `facts` table.  Here is the description of some of the columns:
* `name` - Name of the country
* `area` - Total land and sea area of the country
* `population` - Country's population
* `population_growth` - COuntry's population growth as a percentage
* `birth_rate` - COuntry's birth rate, number of births a year per 1,000 people.
* `death_rate` - COunntry's death rate, number of deaths a year per 1,000 people
* `area_land` - Country's land area in square kilometers
* `area_water` - Country's water area in square kilometers

Let us find the minimum and maximum population and population growth below

In [47]:
%%sql
SELECT MIN(population) as pop_min, MAX(population) as pop_max,
MIN(population_growth) as min_pop_growth, MAX(population_growth) as max_pop_growth FROM facts;

 * sqlite:///factbook.db
Done.


pop_min,pop_max,min_pop_growth,max_pop_growth
0,7256490011,0.0,4.02


The population minimum of zero and the population maximum of 7256490011 stick out, let us look at these more closely

In [48]:
%%sql
SELECT * FROM facts
WHERE population ==(SELECT MIN(population) FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


In [46]:
%%sql
SELECT * FROM facts
WHERE population == (SELECT MAX(population) FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


We can see here that the table contains a row for the whole world.

From the [Antarctica page](https://www.cia.gov/library/publications/the-world-factbook/geos/ay.html) of the CIA World Factbook we find:

'no indigenous inhabitants, but there are both permanent and summer-only staffed research stations

note: 53 countries have signed the 1959 Antarctic Treaty; 30 of those operate through their National Antarctic Program a number of seasonal-only (summer) and year-round research stations on the continent and its nearby islands south of 60 degrees south latitude (the region covered by the Antarctic Treaty); the population engaging in and supporting science or managing and protecting the Antarctic region varies from approximately 4,400 in summer to 1,100 in winter; in addition, approximately 1,000 personnel, including ship's crew and scientists doing onboard research, are present in the waters of the treaty region'

####  Average values for population and area

In [51]:
%%sql
SELECT AVG(population) as avg_pop, AVG(area) as avg_area FROM facts
WHERE name <> 'World';

 * sqlite:///factbook.db
Done.


avg_pop,avg_area
32242666.56846473,555093.546184739


In [54]:
%%sql
SELECT * FROM facts
WHERE population > (SELECT AVG(population) FROM facts)
AND area < (SELECT AVG(area) FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
173,th,Thailand,513120,510890,2230,67976405,0.34,11.19,7.8,0.0
185,uk,United Kingdom,243610,241930,1680,64088222,0.54,12.17,9.35,2.54
192,vm,Vietnam,331210,310070,21140,94348835,0.97,15.96,5.93,0.3


#### Data Exploration
* Which countries have the highest ratios of water to land? Which countries have more water than land?
* Which countries will add the most people to their population next year?
* Which countries have a higher death rate than birth rate?